<a href="https://colab.research.google.com/github/garyhsieh15/RC_design_steel_req/blob/master/LeNet5_tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/work/NCKU/10902/dl/HW04/dl04

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np


(xtrain, ytrain), (xtest, ytest) = tf.keras.datasets.cifar10.load_data()

Instructions for updating:
non-resource variables are not supported in the long term
170500096/170498071 [==============================] - 11s 0us/step


In [ ]:
print("xtrain.shape: \n", xtrain.shape)
print("xtrain: \n", xtrain)
print("xtrain[0].shape: \n", xtrain[0].shape)
print("xtrain[0]: \n", xtrain[0])
print("xtrain[0][0]: \n", xtrain[0][0])
aaa = xtrain.reshape(-1, 32, 32, 3)
bbb = aaa.astype('float32')
print("aaa: \n", aaa[0][0])
print("aaa.shape: \n", aaa.shape)
print("bbb[0][0]: \n", bbb[0][0])
print("bbb[0][0].shape: \n", bbb[0][0].shape)
print("type(aaa) and type(bbb): \n", type(aaa), type(bbb))
#print("xtrain.reshape(-1, 32, 32, 3): \n", xtrain.reshape(-1, 32, 32, 3))

In [ ]:
print("ytrain: \n", ytrain)
print("ytrain.shape: \n", ytrain.shape)
print("ytrain[0]: \n", ytrain[0])
print("ytrain[1]: \n", ytrain[1])

In [ ]:
print("one: \n", xtrain.reshape(-1, 32, 32, 3))
print("two: \n", xtrain.reshape(-1, 32, 32, 3).astype('float32') / 255)

In [ ]:
#import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np



def load_data(num_classes=10):
    (xtrain, ytrain), (xtest, ytest) = tf.keras.datasets.cifar10.load_data()
    xtrain = xtrain.reshape(-1, 32, 32, 3).astype('float32') / 255
    #xtrain = xtrain.reshape(-1, 64, 64, 3).astype('float32') / 255
    xtest = xtest.reshape(-1, 32, 32, 3).astype('float32') / 255
    #xtest = xtest.reshape(-1, 64, 64, 3).astype('float32') / 255
    ytrain = np.eye(num_classes)[ytrain.reshape(-1)] # one hot encoding
    ytest = np.eye(num_classes)[ytest.reshape(-1)]   # one hot encoding
    return xtrain, ytrain, xtest, ytest

def next_batch(batch_size, data, labels):
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:batch_size]
    data_shuffle = [data[i] for i in idx]
    labels_shuffle = [labels[i] for i in idx]
    return np.asarray(data_shuffle), np.asarray(labels_shuffle)


Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
xtrain, ytrain, xtest, ytest = load_data()
print("xtrain.shape: \n", xtrain.shape)
print("ytrain.shape: \n", ytrain.shape)
print("xtest.shape: \n", xtest.shape)
print("ytest.shape: \n", ytest.shape)

170500096/170498071 [==============================] - 4s 0us/step
xtrain.shape: 
 (50000, 32, 32, 3)
ytrain.shape: 
 (50000, 10)
xtest.shape: 
 (10000, 32, 32, 3)
ytest.shape: 
 (10000, 10)


In [ ]:
xtrain, ytrain, xtest, ytest = load_data()

# Parameters
num_epoch = 4000
batch_size = 128

# layer 0: input data
x = tf.placeholder("float", [None,32,32,3])
#x = tf.placeholder("float", [None,64,64,3])
y = tf.placeholder("float", [None,10])

# layer 1: convolution
# filter size = 5x5, input channel = 1, output channel = 32
conv1_w = tf.get_variable("conv1_w", [5,5,3,32], initializer=tf.truncated_normal_initializer(stddev=0.1))
#conv1_w = tf.get_variable("conv1_w", [5,5,3,64], initializer=tf.truncated_normal_initializer(stddev=0.1))
conv1_b = tf.get_variable("conv1_b", [32], initializer=tf.constant_initializer(value=0))
#conv1_b = tf.get_variable("conv1_b", [64], initializer=tf.constant_initializer(value=0))
conv1 = tf.nn.conv2d(x, conv1_w, strides=[1,1,1,1], padding='SAME')
relu1 = tf.nn.relu( tf.nn.bias_add(conv1, conv1_b) )

# layer 2: max pool
# filter size = 2x2, stride = 2
pool1 = tf.nn.max_pool(relu1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

# layer 3: convolution
# filter size = 5x5, input channel = 32, output channel = 64
conv2_w = tf.get_variable("conv2_w", [5,5,32,64], initializer=tf.truncated_normal_initializer(stddev=0.1))
#conv2_w = tf.get_variable("conv2_w", [5,5,64,64], initializer=tf.truncated_normal_initializer(stddev=0.1))
conv2_b = tf.get_variable("conv2_b", [64], initializer=tf.constant_initializer(value=0))
conv2 = tf.nn.conv2d(pool1, conv2_w, strides=[1,1,1,1], padding='SAME')
relu2 = tf.nn.relu( tf.nn.bias_add(conv2, conv2_b) )

# layer 4: max pool
pool2 = tf.nn.max_pool(relu2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

# layer 5: fully connected
fc1_w = tf.get_variable("fc1_w", [8 * 8 * 64, 1024], initializer=tf.truncated_normal_initializer(stddev=0.1))
fc1_b = tf.get_variable("fc1_b", [1024], initializer=tf.constant_initializer(value=0.1))
pool2_vector = tf.reshape(pool2, [-1, 8 * 8 * 64])
fc1 = tf.nn.relu( tf.matmul(pool2_vector, fc1_w) + fc1_b )

# dropout layer
fc1_dropout = tf.nn.dropout(fc1, 1.0)

# layer 6: fully connected
fc2_w = tf.get_variable("fc2_w", [1024, 10], initializer=tf.truncated_normal_initializer(stddev=0.1))
fc2_b = tf.get_variable("fc2_b", [10], initializer=tf.constant_initializer(value=0.1))
y_hat = tf.matmul(fc1_dropout, fc2_w) + fc2_b

# layer 7: softmax, output layer
pred = tf.nn.softmax(y_hat)

# define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=y_hat, labels=y))
optimizer = tf.train.AdamOptimizer()
train_op = optimizer.minimize(loss_op)

# evaluate model
correct = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epoch):
        xbatch, ybatch = next_batch(batch_size, xtrain, ytrain)
        sess.run(train_op, feed_dict={x: xbatch, y: ybatch})

        if ((epoch + 1) % 100 == 0):
            loss, acc = sess.run([loss_op, accuracy], feed_dict={x: xtest, y: ytest})
            print("epoch " + str(epoch+1) + ", loss= " + "{:.4f}".format(loss) + ", acc= " + "{:.3f}".format(acc))

    # Calculate accuracy for MNIST test images
    acc = sess.run(accuracy, feed_dict={x: xtest, y: ytest})
    print('test acc=' + '{:.3f}'.format(acc))

170500096/170498071 [==============================] - 11s 0us/step
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
epoch 100, loss= 1.6028, acc= 0.430
epoch 200, loss= 1.4772, acc= 0.483
epoch 300, loss= 1.3210, acc= 0.528
epoch 400, loss= 1.2645, acc= 0.556
epoch 500, loss= 1.1930, acc= 0.581
epoch 600, loss= 1.1862, acc= 0.589
epoch 700, loss= 1.1220, acc= 0.609
epoch 800, loss= 1.1404, acc= 0.602
epoch 900, loss= 1.1177, acc= 0.619
epoch 1000, loss= 1.0425, acc= 0.646
epoch 1100, loss= 1.0526, acc= 0.644
epoch 1200, loss= 1.0447, acc= 0.651
epoch 1300, loss= 1.0964, acc= 0.641
epoch 1400, loss= 1.0592, acc= 0.659
epoch 1500, loss= 1.0940, acc= 0.650
epoch 1600, loss= 1.0930, acc= 0.657
epoch 1700, loss= 1.0601, acc= 0.672
epoch 1800, loss= 1.0982, acc= 0.668
epoch 1900, loss= 1.0853, acc= 0.662
epoch 2000, loss= 1.1182, acc= 0.668
epoch 2100, loss= 1.1293, acc= 0.664
epoch 2200, loss= 1.1129, acc= 0.672
epoch 2300, 